## DHT Tankers Advanced model

In this model, we improve on our crude excel model by taking into account the already-chartered ships, using data from the TankersInternational twitter feed.

If you can't be bothered to set up tweepy, skip to the fourth cell to use the values I downloaded.

In [1]:
import tweepy
from datetime import datetime, date, time, timedelta
import pickle
import re
import pandas as pd

In [9]:
# Register for a twitter dev account and copy your keys
consumer_key=""
consumer_secret=""
access_token=""
access_token_secret=""

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
auth_api = tweepy.API(auth)

In [ ]:
target="TankersInt"
print("Getting data for " + target)
item = auth_api.get_user(target)
print("name: " + item.name)
print("screen_name: " + item.screen_name)
print("description: " + item.description)
print("statuses_count: " + str(item.statuses_count))
print("friends_count: " + str(item.friends_count))
print("followers_count: " + str(item.followers_count))

# Download recent tweets
stati = []
end_date = datetime.utcnow() - timedelta(days=120)
count = 0
for status in tweepy.Cursor(auth_api.user_timeline, id=target, tweet_mode="extended").items():
    count += 1
    if count % 50 == 0:
        print(count)
    stati.append(status)
    if status.created_at < end_date:
        break
        
with open("dht_twitter_raw.pickle", "wb") as f:
    pickle.dump(stati, f)

In [2]:
with open("dht_twitter_raw.pickle", "rb") as f:
    statuses = pickle.load(f)

In [3]:
dht_statuses = []
for s in statuses:
    if "#" in s.full_text:
        continue
    if "what " in s.full_text.lower():
        continue
    if " of 10" in s.full_text or "10/10" in s.full_text:
        continue
    if s.in_reply_to_status_id is not None:
        continue
    if "DHT" in s.full_text:
        dht_statuses.append((s.created_at, s.full_text))

In [4]:
len(dht_statuses)

30

In [5]:
dht_statuses

[(datetime.datetime(2020, 4, 24, 8, 24, 48),
  'DHT Edelweiss (2008 - Modern VLSFO) On Subs Brazil/China WS 132.50 Repsol May 27-29 Trafigura [Oil Company or Trader Relet] (TCE: RV USD 154.97K @ 78.96 days / Actual USD 152.39K @ 80.12 days) 0.01 idle days'),
 (datetime.datetime(2020, 4, 23, 7, 31, 32),
  'DHT Falcon (2006 - Modern Scrubber) On Subs WAfr/China WS 155.00 Day Harvest May 21-22 DHT Management Demurrage USD 155K (TCE: RV USD 174.41K @ 75.72 days / Actual USD 193.49K @ 68.79 days) 0.75 idle days'),
 (datetime.datetime(2020, 4, 7, 8, 14, 23),
  'DHT Taiga (2012 - Modern Scrubber) On Subs Brazil/China WS 127.50 Petrogal May 1-5 Phillips 66 (TCE: RV USD 147.46K @ 76.92 days / Actual USD 166.29K @ 69.2 days) 0 idle days'),
 (datetime.datetime(2020, 4, 6, 8, 10, 14),
  'DHT Tiger (2017 - Modern VLSFO) On Subs Brazil/China WS 150.00 Petrobras May 2-3 DHT Management (TCE: RV USD 175.81K @ 82.3 days / Actual USD 180.57K @ 80.73 days) 6.05 idle days'),
 (datetime.datetime(2020, 3, 30

In [6]:
data = []
for date, status in dht_statuses:
    if "Failed" in status:
        continue
    row = [date]
    # Ship name
    row.append(re.findall(r"(DHT \w+)", status)[0])
    # Status
    row.append(re.findall(r"\) (.*?) \w+/\w+", status)[0])
    # Source/dest
    trip = re.findall(r"\w+/\w+", status)[0]
    source, dest = trip.split("/")
    row += [source, dest]
    # Start date?
    row.append(re.findall(r"\b(?:Jan?|Feb?|Mar?|Apr?|May?|Jun?|Jul?|Aug?|Sep?|Oct?|Nov?|Dec?) \d{0,2}", status)[0])
    # USD daily rate
    try:
        row.append(float(re.findall(r"Actual USD (\d+.\d+)", status)[0]))
    except IndexError:
        row.append(float(re.findall(r"(\d*\.\d+|\d+)K/day", status)[0]))
    try:
        # Number of days
        row.append(float(re.findall(r"@ (\d+.\d+) days", status)[1]))
    except IndexError:
        continue
    # Idle days
    row.append(float(re.findall(r"([\d+.\d+]) idle days", status)[0]))
    data.append(row)

In [7]:
df = pd.DataFrame(data, columns=["Date", "Ship", "Status", "Source", "Dest", "Start", "Rate", "Days", "IdleDays"])

In [8]:
df.head()

,Date,Ship,Status,Source,Dest,Start,Rate,Days,IdleDays
0,2020-04-24 08:24:48,DHT Edelweiss,On Subs,Brazil,China,May 27,152.39,80.12,1.0
1,2020-04-23 07:31:32,DHT Falcon,On Subs,WAfr,China,May 21,193.49,68.79,5.0
2,2020-04-07 08:14:23,DHT Taiga,On Subs,Brazil,China,May 1,166.29,69.20,0.0
3,2020-04-06 08:10:14,DHT Tiger,On Subs,Brazil,China,May 2,180.57,80.73,5.0
4,2020-03-30 16:58:16,DHT Hawk,On Subs,AG,China,Apr 17,233.24,51.05,6.0


In [9]:
fleet = ["DHT Mustang",
"DHT Bronco",
"DHT Colt",
"DHT Stallion",
"DHT Tiger",
"DHT Puma",
"DHT Panther",
"DHT Lion",
"DHT Leopard",
"DHT Jaguar",
"DHT Opal",
"DHT Sundarbans",
"DHT Taiga",
"DHT Peony",
"DHT Lotus",
"DHT Amazon",
"DHT Redwood",
"DHT Edelweiss",
"DHT China",
"DHT Europe",
"DHT Bauhinia",
"DHT Hawk",
"DHT Scandinavia",
"DHT Falcon",
"DHT Lake",
"DHT Raven",
"DHT Condor",]

table = pd.DataFrame(index=fleet, columns=list(range(365)))

In [10]:
table.head()

,0,1,2,3,4,5,6,7,8,9,...,355,356,357,358,359,360,361,362,363,364
DHT Mustang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DHT Bronco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DHT Colt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DHT Stallion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DHT Tiger,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Chartered as of end 2019
ships = ["DHT " + s for s in {"Amazon", "China", "Sundarbans", "Taiga"}]

# 4 ships, plus lotus for a quarter
rate = 47413/4.25/365
for ship in ships:
    table.loc[ships] = rate

# Treat DHT lotus specially
table.loc["DHT Lotus", 0:90] = rate

In [12]:
# Chartered as of Apr 1: 6 ships @ 67.3k
ships = ["DHT " + s for s in {"Lake", "Raven", "Scandinavia","Bauhinia", "Edelweiss", "Lion"}]
rate = 67.3
start_day = datetime.strptime("Apr 01", "%b %d").timetuple().tm_yday

for ship in ships:
    table.loc[ship, start_day:] = rate

In [13]:
# Pre-existing charters
table.sum(axis=1)

DHT Mustang            0.000000
DHT Bronco             0.000000
DHT Colt               0.000000
DHT Stallion           0.000000
DHT Tiger              0.000000
DHT Puma               0.000000
DHT Panther            0.000000
DHT Lion           18440.200000
DHT Leopard            0.000000
DHT Jaguar             0.000000
DHT Opal               0.000000
DHT Sundarbans     11156.000000
DHT Taiga          11156.000000
DHT Peony              0.000000
DHT Lotus           2781.358904
DHT Amazon         11156.000000
DHT Redwood            0.000000
DHT Edelweiss      18440.200000
DHT China          11156.000000
DHT Europe             0.000000
DHT Bauhinia       18440.200000
DHT Hawk               0.000000
DHT Scandinavia    18440.200000
DHT Falcon             0.000000
DHT Lake           18440.200000
DHT Raven          18440.200000
DHT Condor             0.000000
dtype: float64

In [14]:
# From twitter
for i, row in df.iterrows():
    ship = row["Ship"]
    start_day = datetime.strptime(row["Start"], "%b %d").timetuple().tm_yday
    n_days = int(row["Days"])
    rate = row["Rate"]
    table.loc[ship, start_day:start_day + n_days] = rate

In [15]:
table.sum(axis=1)

DHT Mustang        14815.900000
DHT Bronco             0.000000
DHT Colt               0.000000
DHT Stallion       11056.500000
DHT Tiger          13999.650000
DHT Puma               0.000000
DHT Panther         3898.840000
DHT Lion           18440.200000
DHT Leopard         2861.760000
DHT Jaguar         10820.450000
DHT Opal            7843.080000
DHT Sundarbans     11156.000000
DHT Taiga          20656.793151
DHT Peony           2587.400000
DHT Lotus           5181.409452
DHT Amazon         11156.000000
DHT Redwood         4781.280000
DHT Edelweiss      25952.210000
DHT China          11156.000000
DHT Europe             0.000000
DHT Bauhinia       18440.200000
DHT Hawk           12736.080000
DHT Scandinavia    18440.200000
DHT Falcon         15753.620000
DHT Lake           25026.140000
DHT Raven          18625.190000
DHT Condor             0.000000
dtype: float64

In [16]:
table

,0,1,2,3,4,5,6,7,8,9,...,355,356,357,358,359,360,361,362,363,364
DHT Mustang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DHT Bronco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DHT Colt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DHT Stallion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DHT Tiger,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DHT Puma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DHT Panther,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DHT Lion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,67.3,67.3,67.3,67.3,67.3,67.3,67.3,67.3,67.3,67.3
DHT Leopard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DHT Jaguar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [170]:
# Save to excel
table.fillna(0).to_excel("DHT_tankers_v2.xlsx")